In [1]:
from tvDatafeed import TvDatafeed, Interval
import pandas as pd
import os
import json
import time

In [2]:
# Configuration
username = "truthfulOwl34362"
password = "asdfasdf12#$jkl;"
storage_folder = "D:/Programming/Download_Backtest_Deploy_data/1__Download/TradingView_data_download/storage"
mapping_file = "D:/Programming/Download_Backtest_Deploy_data/1__Download/TradingView_data_download/symbol_mappings.json"

# Load NIFTY 50 symbols
nifty_data = pd.read_csv("D:/Programming/Download_Backtest_Deploy_data/1__Download/1__Download_data_ICICI_via_API/nifty_50_componants_data_from_tradingview.csv")
nifty_50 = nifty_data['Symbol'].values.tolist()

# Create storage folder
os.makedirs(storage_folder, exist_ok=True)

In [3]:
nifty_50 = ['SHREECEM', 'IOC', 'GAIL', 'YESBANK']

In [4]:
# Symbol mapping management
def load_symbol_mappings():
    try:
        with open(mapping_file, 'r') as f:
            return json.load(f)
    except FileNotFoundError:
        return {}

def save_symbol_mappings(mappings):
    with open(mapping_file, 'w') as f:
        json.dump(mappings, f, indent=2)

In [5]:
def download_stock_data(symbol, exchange='NSE', n_bars=1174):
    try:
        tv = TvDatafeed(username=username, password=password)
        data = tv.get_hist(symbol=symbol, exchange=exchange, interval=Interval.in_daily, n_bars=n_bars)
        
        if data is None or data.empty:
            return None
            
        data.reset_index(inplace=True)
        data = data[['datetime', 'open', 'high', 'low', 'close', 'volume']]
        data.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
        data['Date'] = data['Date']
        
        for col in ['Open', 'High', 'Low', 'Close']:
            data[col] = data[col].round(2)
            
        return data
    except:
        return None

In [6]:
# Master download controller
def master_download():
    mappings = load_symbol_mappings()
    successful = []
    failed = []
    
    for i, original_symbol in enumerate(nifty_50, 1):
        print(f"\n[{i}/{len(nifty_50)}] Processing: {original_symbol}")
        
        # Check if file already exists
        filename = f"{storage_folder}/{original_symbol}_historical_data.csv"
        if os.path.exists(filename):
            print(f"Already exists: {original_symbol}")
            successful.append(original_symbol)
            continue
            
        success = False
        
        # Try original symbol first
        data = download_stock_data(original_symbol)
        
        if data is not None:
            data.to_csv(filename, index=False)
            successful.append(original_symbol)
            success = True
        else:
            # Try alternatives from JSON
            if original_symbol in mappings and isinstance(mappings[original_symbol], list):
                for alt_symbol in mappings[original_symbol]:
                    data = download_stock_data(alt_symbol)
                    if data is not None:
                        data.to_csv(filename, index=False)
                        mappings[original_symbol] = alt_symbol
                        save_symbol_mappings(mappings)
                        successful.append(original_symbol)
                        success = True
                        break
            
            # User input for failed symbols
            if not success:
                while True:
                    user_input = input(f"\nSymbol '{original_symbol}' failed. Options:\n1. Enter correct symbol\n2. Skip this symbol\nChoice (1/2): ")
                    
                    if user_input == '2':
                        failed.append(original_symbol)
                        break
                    elif user_input == '1':
                        new_symbol = input(f"Enter correct symbol for {original_symbol}: ").strip().upper()
                        if new_symbol:
                            data = download_stock_data(new_symbol)
                            if data is not None:
                                data.to_csv(filename, index=False)
                                mappings[original_symbol] = new_symbol
                                save_symbol_mappings(mappings)
                                successful.append(original_symbol)
                                print(f"Success with: {new_symbol}")
                                break
                            else:
                                print(f"Failed with: {new_symbol}")
        
        time.sleep(1)
    
    print(f"\n=== Summary ===")
    print(f"Successful: {len(successful)}/{len(nifty_50)}")
    print(f"Failed: {failed}")
    
    return successful, failed

# Run master download
successful, failed = master_download()


[1/4] Processing: SHREECEM


error while signin
you are using nologin method, data you access may be limited



[2/4] Processing: IOC


ERROR:tvDatafeed.main:error while signin



[3/4] Processing: GAIL


ERROR:tvDatafeed.main:error while signin



[4/4] Processing: YESBANK


ERROR:tvDatafeed.main:error while signin



=== Summary ===
Successful: 4/4
Failed: []
